
Extract signals on spheres and plot a connectome
==============================================================

Created by Nilearn-Team

Modified/added by Ralf Schmaelzle, 2020


This example shows how to extract signals from spherical regions.
We show how to build spheres around user-defined coordinates, as well as
centered on coordinates from the Power-264 atlas [1], and the Dosenbach-160
atlas [2].

**References**

[1] Power, Jonathan D., et al. "Functional network organization of the
human brain." Neuron 72.4 (2011): 665-678.

[2] Dosenbach N.U., Nardos B., et al. "Prediction of individual brain maturity
using fMRI.", 2010, Science 329, 1358-1361.

We estimate connectomes using two different methods: **sparse inverse
covariance** and **partial_correlation**, to recover the functional brain
**networks structure**.

We'll start by extracting signals from Default Mode Network regions and
computing a connectome from them.


In [ ]:
import warnings
warnings.filterwarnings('ignore')
from nilearn import plotting
from nilearn.image import mean_img
%matplotlib inline

Retrieve the brain development fmri dataset
-------------------------------------------

We are going to use a subject from the development functional
connectivity dataset.



In [ ]:
from nilearn import datasets
dataset = datasets.fetch_development_fmri(n_subjects=20)

# print basic information on the dataset
print('First subject functional nifti image (4D) is at: %s' %
      dataset.func[0])  # 4D data

Coordinates of Default Mode Network
------------------------------------



In [ ]:
dmn_coords = [(0, -52, 18), (-46, -68, 32), (46, -68, 32), (1, 50, -5)]
labels = [
    'Posterior Cingulate Cortex',
    'Left Temporoparietal junction',
    'Right Temporoparietal junction',
    'Medial prefrontal cortex',
    ]

***Exercise:***

Go to an online-brain viewer of your choice and look up one of these coordinates

Extracts signal from sphere around DMN seeds
----------------------------------------------

We can compute the mean signal within **spheres** of a fixed radius
around a sequence of (x, y, z) coordinates with the object
:class:`nilearn.input_data.NiftiSpheresMasker`.
The resulting signal is then prepared by the masker object: Detrended,
band-pass filtered and **standardized to 1 variance**.



In [ ]:
from nilearn import input_data

masker = input_data.NiftiSpheresMasker(
    dmn_coords, radius=8,
    detrend=True, standardize=True,
    low_pass=0.1, high_pass=0.01, t_r=2,
    memory='nilearn_cache', memory_level=1, verbose=2)

# Additionally, we pass confound information to ensure our extracted
# signal is cleaned from confounds.

func_filename = dataset.func[0]
confounds_filename = dataset.confounds[0]

time_series = masker.fit_transform(func_filename,
                                   confounds=[confounds_filename])

***Exercise:***

Insert a new cell and use it to explore the time_series array? 

What is its shape? Why is it that size?

Other ideas?

Display time series
--------------------



In [ ]:
import matplotlib.pyplot as plt

for time_serie, label in zip(time_series.T, labels):
    plt.plot(time_serie, label=label)

plt.title('Default Mode Network Time Series')
plt.xlabel('Scan number')
plt.ylabel('Normalized signal')
plt.legend()
plt.tight_layout()

Compute  correlation matrix
-----------------------------------

In [ ]:
from nilearn.connectome import ConnectivityMeasure
connectivity_measure = ConnectivityMeasure(kind='correlation')
correlation_matrix = connectivity_measure.fit_transform(
    [time_series])[0]

***Exercise:***

Insert a new cell and use it to explore the correlation_matrix array? 

What is its shape? Why is it that size?

Other ideas?

Display connectome
-------------------

We display the graph of connections with
`:func: nilearn.plotting.plot_connectome`.



In [ ]:
from nilearn import plotting

plotting.plot_connectome(correlation_matrix, dmn_coords,
                         title="Default Mode Network Connectivity")

Display connectome with hemispheric projections.
Notice (0, -52, 18) is included in both hemispheres since x == 0.



In [ ]:
plotting.plot_connectome(correlation_matrix, dmn_coords,
                         title="Connectivity projected on hemispheres",
                         display_mode='lyrz')

plotting.show()

3D visualization in a web browser
---------------------------------
An alternative to :func:`nilearn.plotting.plot_connectome` is to use
:func:`nilearn.plotting.view_connectome`, which gives more interactive
visualizations in a web browser. See `interactive-connectome-plotting`
for more details.



In [ ]:
view = plotting.view_connectome(correlation_matrix, dmn_coords)

# In a Jupyter notebook, if ``view`` is the output of a cell, it will
# be displayed below the cell
view

Extract signals on spheres from an atlas
----------------------------------------

Next, instead of supplying our own coordinates, we will use coordinates
generated at the center of mass of regions from two different atlases.
This time, we'll use a different correlation measure.

First we fetch the coordinates of the Power atlas



In [ ]:
power = datasets.fetch_coords_power_2011()
print('Power atlas comes with {0}.'.format(power.keys()))

Compute within spheres averaged time-series
-------------------------------------------

We collect the regions coordinates in a numpy array



In [ ]:
import numpy as np

coords = np.vstack((power.rois['x'], power.rois['y'], power.rois['z'])).T

print('Stacked power coordinates in array of shape {0}.'.format(coords.shape))

and define spheres masker, with small enough radius to avoid regions overlap.



In [ ]:
spheres_masker = input_data.NiftiSpheresMasker(
    seeds=coords, smoothing_fwhm=6, radius=5.,
    detrend=True, standardize=True, low_pass=0.1, high_pass=0.01, t_r=2)

timeseries = spheres_masker.fit_transform(func_filename,
                                          confounds=confounds_filename)

Estimate correlations
---------------------

We start by estimating the signal **covariance** matrix. Here the
number of ROIs exceeds the number of samples,



In [ ]:
print('time series has {0} samples'.format(timeseries.shape[0]))

in which situation the graphical lasso **sparse inverse covariance**
estimator captures well the covariance **structure**.



In [ ]:
try:
    from sklearn.covariance import GraphicalLassoCV
except ImportError:
    # for Scitkit-Learn < v0.20.0
    from sklearn.covariance import GraphLassoCV as GraphicalLassoCV

covariance_estimator = GraphicalLassoCV(cv=3, verbose=1)

We just fit our regions signals into the `GraphicalLassoCV` object



In [ ]:
covariance_estimator.fit(timeseries)

and get the ROI-to-ROI covariance matrix.



In [ ]:
matrix = covariance_estimator.covariance_
print('Covariance matrix has shape {0}.'.format(matrix.shape))

Plot matrix, graph, and strength
--------------------------------

We use `:func: nilearn.plotting.plot_matrix` to visualize our correlation matrix
and display the graph of connections with `nilearn.plotting.plot_connectome`.



In [ ]:
from nilearn import plotting

plotting.plot_matrix(matrix, vmin=-1., vmax=1., colorbar=True,
                     title='Power correlation matrix')

# Tweak edge_threshold to keep only the strongest connections.
plotting.plot_connectome(matrix, coords, title='Power correlation graph',
                         edge_threshold='99.8%', node_size=20, colorbar=True)

Sometimes, the information in the correlation matrix is overwhelming and
aggregating edge strength from the graph would help. Use the function
`nilearn.plotting.plot_connectome_strength` to visualize this information.



In [ ]:
plotting.plot_connectome_strength(
    matrix, coords, title='Connectome strength for Power atlas'
)

From the correlation matrix, we observe that there is a positive and negative
structure. We could make two different plots by plotting these strengths
separately.



In [ ]:
from matplotlib.pyplot import cm

# plot the positive part of of the matrix
plotting.plot_connectome_strength(
    np.clip(matrix, 0, matrix.max()), coords, cmap=cm.YlOrRd,
    title='Strength of the positive edges of the Power correlation matrix'
)

# plot the negative part of of the matrix
plotting.plot_connectome_strength(
    np.clip(matrix, matrix.min(), 0), coords, cmap=cm.PuBu,
    title='Strength of the negative edges of the Power correlation matrix'
)

Connectome extracted from Dosenbach's atlas
-------------------------------------------

We repeat the same steps for Dosenbach's atlas.



In [ ]:
dosenbach = datasets.fetch_coords_dosenbach_2010()

coords = np.vstack((
    dosenbach.rois['x'],
    dosenbach.rois['y'],
    dosenbach.rois['z'],
)).T

spheres_masker = input_data.NiftiSpheresMasker(
    seeds=coords, smoothing_fwhm=6, radius=4.5,
    detrend=True, standardize=True, low_pass=0.1, high_pass=0.01, t_r=2)

timeseries = spheres_masker.fit_transform(func_filename,
                                          confounds=confounds_filename)

covariance_estimator = GraphicalLassoCV()
covariance_estimator.fit(timeseries)
matrix = covariance_estimator.covariance_

plotting.plot_matrix(matrix, vmin=-1., vmax=1., colorbar=True,
                     title='Dosenbach correlation matrix')

plotting.plot_connectome(matrix, coords, title='Dosenbach correlation graph',
                         edge_threshold="99.7%", node_size=20, colorbar=True)
plotting.plot_connectome_strength(
    matrix, coords, title='Connectome strength for Power atlas'
)
plotting.plot_connectome_strength(
    np.clip(matrix, 0, matrix.max()), coords, cmap=cm.YlOrRd,
    title='Strength of the positive edges of the Power correlation matrix'
)
plotting.plot_connectome_strength(
    np.clip(matrix, matrix.min(), 0), coords, cmap=cm.PuBu,
    title='Strength of the negative edges of the Power correlation matrix'
)

We can easily identify the Dosenbach's networks from the matrix blocks.



In [ ]:
print('Dosenbach networks names are {0}'.format(np.unique(dosenbach.networks)))

plotting.show()